# SVM Training on MNIST

In this notebook, we are going to train an L2-SVM model on PYNQ using stochasting gradient descent optimization, to classify handwritten digits of the MNIST data set. We are going to perform training on both the CPU and the FPGA, comparing performance.

## 1. Instantiate ZipML_SGD

First, we instantiate a ZipML_SGD object, that we will use for training and inference.

In [1]:
import zipml
import numpy as np
import time

Z = zipml.ZipML_SGD(on_pynq=1, bitstreams_path=zipml.BITSTREAMS, ctrl_base=0x41200000, dma_base=0x40400000, dma_buffer_size=32*1024*1024)

floatFSGD.bit is loaded
Got sgd_ctrl!
Got sgd_dma!
Allocated buffer of size: 33554432 bytes


## 2. Load and prepare the data

We load the data that is formatted in libsvm format (label feature_index1:feature1 feature_index2:feature2 ...).
Then, we perform a normalization on the features of the data set, where we can specify: (1) The normalization range (-1 to 1 or 0 to 1), (2) if the normalization should happen row-wise or column-wise.

In [2]:
!./get_mnist.sh

[mnist] Nothing to do.
[mnist.t] Nothing to do.


In [3]:
start = time.time()
Z.load_libsvm_data('./mnist/mnist', 10000, 784)
print('Data loaded, time: ' + str(time.time()-start) )
Z.a_normalize(to_min1_1=0, row_or_column='r') # Normalize features of the data set

# Set training related parameters
num_epochs = 10
step_size = 1.0/(1 << 12)
cost_pos = 1.0
cost_neg = 1.5

num_values_one_row: 788
Data loaded, time: 33.774189949035645


Let's visualize our date set, to take a look at the handwritten digits and to make sure if we have loaded it correctly. 

In [4]:
import scipy.misc
for i in range(0,10):
    scipy.misc.toimage( np.reshape(Z.a[i,1:785], (28,28)) ).save('out'+ str(i) +'.jpg')

from PIL import Image
from IPython.display import display

for i in range(0,10):
    im = Image.open('./out'+str(i)+'.jpg')
    display(im)

## 3. Full precision training using the CPU

Let's perform training on full-precision data using the CPU. With the mnist data set we want to classify between 10 classes (handwritten digits from 0 to 9). For this reason, we perform a 1-vs-all training: For each digit we train a seperate L2-SVM model, that should classify the digit compared to all others.

During the training, we calculate the loss after each epoch (a complete scan over the data set). The convergence of the model is observed by the decreasing loss. After having performed the given number of epochs, we save the trained model in an array.

In [5]:
xs_CPU = np.zeros((Z.num_features, 10))
for c in range(0, 10):
	start = time.time()
	Z.b_binarize(c) # Binarize the labels of the data set
	print('b binarized for ' + str(c) + ", time: " + str(time.time()-start) )
	start = time.time()
    
    # Train model on the CPU
	x_history = Z.L2SVM_SGD(num_epochs, step_size, cost_pos, cost_neg, regularize=0, use_binarized=1)
    
	print('Training time: ' + str(time.time()-start) )
    # Print losses after each epoch
	initial_loss = Z.calculate_L2SVM_loss(np.zeros(Z.num_features), cost_pos, cost_neg, 0, 1)
	print('Initial loss: ' + str(initial_loss))
	for e in range(0, num_epochs):
		loss = Z.calculate_L2SVM_loss(x_history[:,e], cost_pos, cost_neg, 0, 1)
		print('Epoch ' + str(e) + ' loss: ' + str(loss) )

	xs_CPU[:,c] = x_history[:,num_epochs-1]

b binarized for 0, time: 0.4593064785003662
Training time: 21.559975147247314
Initial loss: 7249.75
Epoch 0 loss: 407.051758304
Epoch 1 loss: 322.004559011
Epoch 2 loss: 282.7098222
Epoch 3 loss: 258.917717091
Epoch 4 loss: 242.779521611
Epoch 5 loss: 230.881771687
Epoch 6 loss: 221.562047428
Epoch 7 loss: 214.064932173
Epoch 8 loss: 207.702874743
Epoch 9 loss: 202.129375769
b binarized for 1, time: 0.4598066806793213
Training time: 21.79761576652527
Initial loss: 7218.25
Epoch 0 loss: 428.357593619
Epoch 1 loss: 345.685836409
Epoch 2 loss: 312.849864526
Epoch 3 loss: 294.124601755
Epoch 4 loss: 281.348371555
Epoch 5 loss: 271.739573022
Epoch 6 loss: 264.078599304
Epoch 7 loss: 257.732066977
Epoch 8 loss: 252.334974374
Epoch 9 loss: 247.658311389
b binarized for 2, time: 0.4562089443206787
Training time: 25.91661763191223
Initial loss: 7252.25
Epoch 0 loss: 764.145994427
Epoch 1 loss: 645.997702502
Epoch 2 loss: 598.404771318
Epoch 3 loss: 570.738883041
Epoch 4 loss: 552.038573278
Epoc

## 4. Full-precision training using the FPGA

Now, let's use the ZipML-PYNQ overlay to perform the same training process on the FPGA. We just replace the CPU-training function with the FPGA one.

In [6]:
xs_floatFSGD = np.zeros((Z.num_features, 10))
for c in range(0, 10):
	start = time.time()
	Z.b_binarize(c)
	print('b binarized for ' + str(c) + ", time: " + str(time.time()-start) )
	start = time.time()
	
    # Train model on the FPGA
	Z.configure_SGD_FPGA(num_epochs, step_size, cost_pos, cost_neg, 1, c)
	x_history = Z.SGD_FPGA(num_epochs)

	print('Training time: ' + str(time.time()-start) )
    # Print losses after each epoch
	initial_loss = Z.calculate_L2SVM_loss(np.zeros(Z.num_features), cost_pos, cost_neg, 0, 1)
	print('Initial loss: ' + str(initial_loss))
	for e in range(0, num_epochs):
		loss = Z.calculate_L2SVM_loss(x_history[:,e], cost_pos, cost_neg, 0, 1)
		print('Epoch ' + str(e) + ' loss: ' + str(loss) )

	xs_floatFSGD[:,c] = x_history[:,num_epochs-1]

b binarized for 0, time: 0.452847957611084
Sent
Config Received
0x20
0x20000
0x39c00000
0x39800000
0x0
0x311
0x2710
0xa
0x39800000
self.data_start_index: 0, self.data_end_index: 7880000
bytes_for_model: 3144
Training time: 0.5656685829162598
Initial loss: 7249.75
Epoch 0 loss: 407.21395011
Epoch 1 loss: 322.181103171
Epoch 2 loss: 282.834546258
Epoch 3 loss: 259.025670617
Epoch 4 loss: 242.873300504
Epoch 5 loss: 230.962467829
Epoch 6 loss: 221.631189245
Epoch 7 loss: 214.125792615
Epoch 8 loss: 207.756877901
Epoch 9 loss: 202.17128729
b binarized for 1, time: 0.4604675769805908
Sent
Config Received
0x20
0x20000
0x39c00000
0x39800000
0x3f800000
0x311
0x2710
0xa
0x39800000
self.data_start_index: 0, self.data_end_index: 7880000
bytes_for_model: 3144
Training time: 0.5233240127563477
Initial loss: 7218.25
Epoch 0 loss: 428.428984678
Epoch 1 loss: 345.724257876
Epoch 2 loss: 312.884168104
Epoch 3 loss: 294.155814688
Epoch 4 loss: 281.37468518
Epoch 5 loss: 271.76528027
Epoch 6 loss: 264.10

The training time on the FPGA is approximately 0.6 seconds for one digit, whereas on the CPU it was 20 secons. 2 orders of magnitude speedup!

## 5. Low-precision training using the FPGA

With the ZipML-PYNQ overlay, we can even improve upon previous results by using low-precision data. We call the quantization function that compresses the features of the data set using deterministic quantization. The rest of the code remains exactly the same as the previous example. We can see that the total training time is improved X times.

In [7]:
# Quantize the features of the data set
Z.a_quantize(quantization_bits=1)

xs_qFSGD = np.zeros((Z.num_features, 10))
for c in range(0, 10):
	start = time.time()
	Z.b_binarize(c)
	print('b binarized for ' + str(c) + ", time: " + str(time.time()-start) )
	start = time.time()
	
    # Train model on the FPGA
	Z.configure_SGD_FPGA(num_epochs, step_size, cost_pos, cost_neg, 1, c)
	x_history = Z.SGD_FPGA(num_epochs)

	print('Training time: ' + str(time.time()-start) )
    # Print losses after each epoch
	initial_loss = Z.calculate_L2SVM_loss(np.zeros(Z.num_features), cost_pos, cost_neg, 0, 1)
	print('Initial loss: ' + str(initial_loss))
	for e in range(0, num_epochs):
		loss = Z.calculate_L2SVM_loss(x_history[:,e], cost_pos, cost_neg, 0, 1)
		print('Epoch ' + str(e) + ' loss: ' + str(loss) )

	xs_qFSGD[:,c] = x_history[:,num_epochs-1]

num_levels: 2
qFSGD1.bit is loaded
num_quantized_items_in_word: 32.0
mask: 1
self.total_size: 280000
b binarized for 0, time: 0.45632052421569824
Sent
Config Received
0x20
0x20000
0x39c00000
0x39800000
0x0
0x311
0x2710
0xa
0x39800000
self.data_start_index: 7880000, self.data_end_index: 8160000
bytes_for_model: 7424
Training time: 0.14095163345336914
Initial loss: 7249.75
Epoch 0 loss: 405.620035875
Epoch 1 loss: 318.464490402
Epoch 2 loss: 279.054448444
Epoch 3 loss: 255.509053215
Epoch 4 loss: 239.283327675
Epoch 5 loss: 227.132768047
Epoch 6 loss: 217.552909051
Epoch 7 loss: 209.784909852
Epoch 8 loss: 203.352229473
Epoch 9 loss: 197.871875076
b binarized for 1, time: 0.4523048400878906
Sent
Config Received
0x20
0x20000
0x39c00000
0x39800000
0x3f800000
0x311
0x2710
0xa
0x39800000
self.data_start_index: 7880000, self.data_end_index: 8160000
bytes_for_model: 7424
Training time: 0.14049386978149414
Initial loss: 7218.25
Epoch 0 loss: 434.454025393
Epoch 1 loss: 353.236780034
Epoch 2 los

With quantized data (here we used 1-bit) we can accelerate training even more. Here, the training for 1 digit takes approximately 0.14 seconds. The reason why we can't observe the theoretical limit of 32x speedup (1-bit vs single-precision floating-point) here is the DMA initiation overheads and the fact that we are transferring the whole model (remember, the model is still in full precision) back after each epoch.

At this point, go ahead and test the trained models that you have trained with low precision data. You are going to observe that you still reach 90% classification accuracy!

## Testing the trained models

Now that we have trained our models for each digit, we can call the classification function and see how it performs.

In [9]:
Z.load_libsvm_data('./mnist/mnist.t', 10000, 784)
Z.a_normalize(to_min1_1=0, row_or_column='r');

classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
matches_CPU = Z.multi_classification(xs_CPU, classes)
matches_floatFSGD = Z.multi_classification(xs_floatFSGD, classes)
matches_qFSGD = Z.multi_classification(xs_qFSGD, classes)

print('CPU training -> Matches ' + str(matches_CPU) + ' out of ' + str(Z.num_samples) + ' samples.')
print('FPGA training -> Matches ' + str(matches_floatFSGD) + ' out of ' + str(Z.num_samples) + ' samples.')
print('1-bit FPGA training -> Matches ' + str(matches_qFSGD) + ' out of ' + str(Z.num_samples) + ' samples.')

num_values_one_row: 788
CPU training -> Matches 9002 out of 10000 samples.
FPGA training -> Matches 9003 out of 10000 samples.
1-bit FPGA training -> Matches 9000 out of 10000 samples.
